In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook, we show how a **DFA** (Deterministic Finite Automaton) can be minimized by identifying states that are equivalent.
We will strictly adhere to the type definitions established in the previous notebooks.

## Data Structures

We import our strict types. Since `key` is defined generically over `Value` (from `recursive-set`), it can handle both original states and minimized states (sets of states) without modification.

In [ ]:
import { RecursiveSet, RecursiveMap, Tuple, Value } from "recursive-set";
import { DFA, DFAState, Char, TransRelDet } from "./01-NFA-2-DFA";

## Type Definitions

We define the types for the **Quotient Automaton** (the minimized DFA).

* **`MinState`**: A state in the minimized DFA (Level 3: Set of equivalent `DFAState`s).
* **`Pair`**: A tuple $(p, q)$ of two `DFAState`s.
* **`MinDFA`**: The structure of the minimized automaton.

In [ ]:
type MinState = RecursiveSet<DFAState>; 
type MinTransRel = RecursiveMap<Tuple<[MinState, Char]>, MinState>;
type Pair = Tuple<[DFAState, DFAState]>;

type MinDFA = {
    Q: RecursiveSet<MinState>;
    Σ: RecursiveSet<Char>;
    δ: MinTransRel;
    q0: MinState;
    A: RecursiveSet<MinState>;
}

### Helper Functions

**Arbitrary Element:**
The function `arb(M)` returns an *arbitrary* element from a set `M`.
In our implementation using `recursive-set`, elements are internally sorted by their **hash code** for efficiency. Therefore, this function deterministically returns the element with the smallest hash. While this is not necessarily the "smallest" value (numerically or lexicographically), it ensures a consistent canonical representative for any given set.

In [ ]:
function arb<T extends Value>(M: RecursiveSet<T>): T {
    for (const e of M) return e;
    throw new Error("Unreachable");
}

### Helper: Finding Separable Pairs

The function `separate` identifies pairs of states $(q_1, q_2)$ that can be distinguished by a single transition step into an already known distinguishable pair.

It takes four arguments:
* `Pairs`: A set of state pairs $(p_1, p_2)$ that are *already known* to be separable.
* `States`: The set of all states in the DFA.
* `Σ`: The alphabet.
* `δ`: The transition function.

**Logic:**
A new pair $(q_1, q_2)$ is added to the result if there exists **some** character $c \in \Sigma$ such that the transition targets $(p_1, p_2) = (\delta(q_1, c), \delta(q_2, c))$ are already in the `Pairs` set.

$$\text{Result} = \{ (q_1, q_2) \mid \exists c \in \Sigma : (\delta(q_1, c), \delta(q_2, c)) \in \text{Pairs} \}$$

In [ ]:
function separate( Pairs: RecursiveSet<Pair>, States: RecursiveSet<DFAState>, Σ: RecursiveSet<Char>, δ: TransRelDet): RecursiveSet<Pair> {
    const next = new RecursiveSet<Pair>();
    for (const q1 of States) for (const q2 of States) {
        const isSeparable = [...Σ].some(c => {
            const p1 = δ.get(new Tuple(q1, c));
            const p2 = δ.get(new Tuple(q2, c));
            return p1 && p2 && Pairs.has(new Tuple(p1, p2));
        });
        if (isSeparable) next.add(new Tuple(q1, q2));
    }
    return next;
}

### Equivalence Class Lookup

Given a state `p` and a partition of the states (a set of sets), this function returns the specific set (equivalence class) that contains `p`.

In [ ]:
function findEquivalenceClass(p: DFAState, Partition: RecursiveSet<MinState>): MinState {
    for (const C of Partition) if (C.has(p)) return C;
    throw new Error(`State ${p} not found in partition`);
}

### Computing All Separable Pairs

The function `allSeparable` computes the set of all distinguishable state pairs using a fixed-point iteration (Moore's Algorithm).



1.  **Initialization:** We start with pairs that are *trivially distinguishable*: one is accepting ($A$), the other is not ($Q \setminus A$).
    $$\text{Separable}_0 = ((Q \setminus A) \times A) \cup (A \times (Q \setminus A))$$
2.  **Iteration:** We repeatedly call `separate` to find new pairs that lead to already separable pairs via a transition.
3.  **Termination:** We stop when no new pairs are found (fixed point reached: $\text{New} \subseteq \text{Current}$).

In [ ]:
function allSeparable( Q: RecursiveSet<DFAState>, A: RecursiveSet<DFAState>, Σ: RecursiveSet<Char>, δ: TransRelDet ): RecursiveSet<Pair> {
    const NonA = Q.difference(A);
    let separable = NonA.cartesianProduct(A).union(A.cartesianProduct(NonA));
    while (true) {
        const next = separate(separable, Q, Σ, δ);
        if (next.isSubset(separable)) return separable;
        separable = separable.union(next);
    }
}

### Reachability Analysis

The function `reachable` filters out states that cannot be reached from the start state $q_0$. Unreachable states are irrelevant for the language accepted by the DFA and must be removed before minimization.

We provide two implementations to illustrate the difference between the algorithmic and the mathematical perspective:

The implementation directly mirrors the set-theoretical definition found in textbooks. It computes the set of reachable states as a **Fixed-Point Iteration**:
    * Start with $R_0 = \{q_0\}$.
    * Iteratively add all direct successors: $R_{i+1} = R_i \cup \{ \delta(q, c) \mid q \in R_i, c \in \Sigma \}$.
    * Stop when the set stabilizes (i.e., $R_{i+1} \subseteq R_i$).

In [ ]:
function reachable( q0: DFAState, Σ: RecursiveSet<Char>, δ: TransRelDet ): RecursiveSet<DFAState> {
    let result = new RecursiveSet<DFAState>(q0);
    while (true) {
        const next = new RecursiveSet<DFAState>();
        for (const p of result) for (const c of Σ) {
            const target = δ.get(new Tuple(p, c));
            if (target) next.add(target);
        }
        if (next.isSubset(result)) return result;
        result = result.union(next);
    }
}

### DFA Minimization: The Algorithm

The function `minimize` computes the minimal DFA $F_{min}$ for a given input DFA $F = (Q, \Sigma, \delta, q_0, A)$.

The algorithm proceeds in **four mathematical steps**:

#### 1. Elimination of Unreachable States
First, we restrict the state set to those reachable from the start state $q_0$.
$$Q_{reach} = \texttt{reachable}(q_0, \Sigma, \delta)$$
Accordingly, we restrict the set of accepting states:
$$A_{reach} = Q_{reach} \cap A$$

#### 2. Computation of Separable Pairs
We identify all pairs of states that can be distinguished by some input string $w$.
$$\texttt{Separable} = \texttt{allSeparable}(Q_{reach}, A_{reach}, \Sigma, \delta)$$

#### 3. Construction of Equivalence Classes
States that are *not* separable are considered **equivalent**.
$$\texttt{Equivalent} = (Q_{reach} \times Q_{reach}) \setminus \texttt{Separable}$$

We group these states into equivalence classes (blocks), forming a partition of the state space:
$$\texttt{Partition} = \{ [q] \mid q \in Q_{reach} \}$$
where $[q] = \{ p \in Q_{reach} \mid (p, q) \in \texttt{Equivalent} \}$.

#### 4. Construction of the Quotient Automaton
We construct the new minimal DFA $F_{min} = (\mathcal{Q}, \Sigma, \Delta, \mathcal{q}_0, \mathcal{A})$ where:
* $\mathcal{Q} = \texttt{Partition}$ (The states are the blocks).
* $\mathcal{q}_0 = [q_0]$ (The block containing the original start state).
* $\mathcal{A} = \{ C \in \texttt{Partition} \mid \text{arb}(C) \in A_{reach} \}$ (Blocks containing accepting states).
* The transition function $\Delta$ is defined via representatives:
    $$\Delta(C, c) = [\delta(\text{arb}(C), c)]$$

In [ ]:
function minimize(F: DFA): MinDFA {
    const Q = reachable(F.q0, F.Σ, F.δ);
    const A = Q.intersection(F.A); 

    const Separable = allSeparable(Q, A, F.Σ, F.δ);
    const Equivalent = Q.cartesianProduct(Q).difference(Separable);

    const Partition = new RecursiveSet<MinState>();
    for (const q of Q) {
        const equivalentStates = [...Q].filter(p => Equivalent.has(new Tuple(p, q)));
        Partition.add(new RecursiveSet<DFAState>(...equivalentStates));
    }

    const newQ0 = findEquivalenceClass(F.q0, Partition);
    
    const validBlocks = [...Partition].filter(C => A.has(arb(C)));
    const newA = new RecursiveSet<MinState>(...validBlocks);

    const newDelta = new RecursiveMap<Tuple<[MinState, Char]>, MinState>();
    for (const C of Partition) {
        const rep = arb(C);
        for (const c of F.Σ) {
            const target = F.δ.get(new Tuple(rep, c));
            if (target && Q.has(target)) {
                const targetBlock = findEquivalenceClass(target, Partition);
                newDelta.set(new Tuple(C, c), targetBlock);
            }
        }
    }

    return { Q: Partition, Σ: F.Σ, δ: newDelta, q0: newQ0, A: newA };
}

The notebook `08-Test-Minimize.ipynb` provides a test for the function `minimize`.